# 🎙️ Audio to Article Notebook
This notebook lets you:
1. Upload an audio file,
2. Transcribe speech to text (using Whisper),
3. Draft an article from the transcript (via API),
4. Improve the article quality (summarization),
5. Export the article to **DOCX** and **PDF**,
6. Download the exported files directly inside the notebook.

### Install Required Libraries

In [ ]:
# %pip install git+https://github.com/openai/whisper.git torch python-docx reportlab google-genai ipywidgets

## 📥 Import Libraries

In [ ]:
import os

import whisper
from docx import Document
from google import genai
from google.genai import types
from IPython.display import HTML, display
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph, SimpleDocTemplate

## 🎧 Step 1: Transcribe Audio

In [ ]:
def transcribe_audio(audio_path: str) -> str:
    model = whisper.load_model("small")  # choose: tiny, base, small, medium, large
    result = model.transcribe(audio_path)
    return result["text"]


project_path = (
    "D:\\Education_Profession\\Projects\\Personal_Projects\\audio_to_article\\"
)
file_name = "AI_Foundation_Model_Selection.mp3"
audio_file_path = project_path + "audio_files\\" + file_name

file_name_without_extension, file_extension = os.path.splitext(file_name)
transcript_path = (
    project_path + "\\transcripts\\" + file_name_without_extension + "_transcript.txt"
)
if os.path.isfile(transcript_path):
    print("The transcript is already extracted and saved.")
    # load the transcript (optional)
    with open(transcript_path, "r", encoding="utf-8") as file:
        transcript = file.read()
    print("The transcript is loaded and ready.")
else:
    transcript = transcribe_audio(audio_file_path)
    # save the transcript (optional)
    with open(transcript_path, "w", encoding="utf-8") as file:
        file.write(transcript)
    print(f"Transcription is extracted and saved to {transcript_path}")

d:\Education_Profession\Projects\Personal_Projects\audio_to_article\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription is extracted and saved to D:\Education_Profession\Projects\Personal_Projects\audio_to_article\\transcripts\AI_Foundation_Model_Selection_transcript.txt


### Review the Trascription

In [ ]:
# verifying the draft

# This CSS uses VS Code's theme variables for colors,
# with fallback values for other environments like standard Jupyter.
html_code = f"""
<div style="
    max-height: 300px;
    overflow-y: scroll;
    border: 1px solid var(--vscode-contrastBorder, #e0e0e0);
    padding: 10px;
    color: var(--vscode-editor-foreground, #000000);
    background-color: var(--vscode-editor-background, #ffffff);
">
    {transcript}
</div>
"""

display(HTML(html_code))

## ✍️ Step 2: Draft Article

In [ ]:
google_api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=google_api_key)


def draft_article(transcript: str, foundation_model: str, temperature: float = 0.5):
    try:
        article_draft = client.models.generate_content(
            model=foundation_model,
            config=types.GenerateContentConfig(temperature=temperature),
            contents=f"""Draft an informative, well-structured article with three to five paragraphs based on the following transcript: {transcript}""",
        )
        article_draft_text = article_draft.text
        return article_draft_text

    except Exception as e:
        # The function will now handle errors gracefully instead of crashing.
        print(f"An error occurred during article generation: {e}")
        return None

In [ ]:
article_draft_text = draft_article(
    transcript=transcript, foundation_model="gemini-2.5-flash", temperature=0.4
)

### Review the article

In [ ]:
html_code = f"""
<div style="
    max-height: 300px;
    overflow-y: scroll;
    border: 1px solid var(--vscode-contrastBorder, #e0e0e0);
    padding: 10px;
    color: var(--vscode-editor-foreground, #000000);
    background-color: var(--vscode-editor-background, #ffffff);
">
    {article_draft_text}
</div>
"""

display(HTML(html_code))

## 📝 Step 3: Improve Article

In [ ]:
improved_article = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"Improve the following article for grammar, clarity, and readability without shortening it. \
        Keep the original title and all section headings exactly as they appear. \
        Return only the improved article text — no introductions, explanations, or additional commentary:\n\n{article_draft_text}",
)

### Review the improved article

In [ ]:
improved_article_text = improved_article.text
html_code = f"""
<div style="
    max-height: 300px;
    overflow-y: scroll;
    border: 1px solid var(--vscode-contrastBorder, #e0e0e0);
    padding: 10px;
    color: var(--vscode-editor-foreground, #000000);
    background-color: var(--vscode-editor-background, #ffffff);
">
    {improved_article_text}
</div>
"""

display(HTML(html_code))

## 💾 Step 4: Export Functions

In [ ]:
def export_to_docx(text: str, filename: str):
    doc = Document()
    doc.add_paragraph(text)
    doc.save(filename)


def export_to_pdf(text: str, filename: str):
    """
    The corrected function using Platypus Paragraphs for proper text wrapping.
    """
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    formatted_text = text.replace("\n", "<br/>")
    p = Paragraph(formatted_text, styles["Normal"])
    doc.build([p])


save_directory = project_path + "articles\\"
file_name = file_name_without_extension + "_article"
save_article_path = os.path.join(save_directory, file_name)

In [ ]:
# Export the article as a MS Word file
export_to_docx(improved_article_text, save_article_path + ".docx")
print(f"The Article is saved correctly to: {save_article_path} as a MS Word file.")

The Article is saved correctly to: D:\Education_Profession\Projects\Personal_Projects\audio_to_article\articles\AI_Foundation_Model_Selection_article as a MS Word file.


In [ ]:
export_to_pdf(improved_article_text, save_article_path + ".pdf")
print(f"The article is saved correctly to: {save_article_path} as a PDF file.")

The article is saved correctly to: D:\Education_Profession\Projects\Personal_Projects\audio_to_article\articles\AI_Foundation_Model_Selection_article as a PDF file.
